In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images
print(train_features.shape)
print(valid_features.shape)
print(test_features.shape)
print(type(train_features))

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1,k,k,1], strides = [1,k,k,1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k = 2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k = 2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28,28,1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

#Model
logits = conv_net(x, weights, biases, keep_prob)

#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.
            })
            
        print('Epoch {:>2},Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch + 1, loss, valid_acc))
            
     # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))        

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz
(55000, 28, 28, 1)
(5000, 28, 28, 1)
(10000, 28, 28, 1)
<class 'numpy.ndarray'>
Epoch  1,Loss:  1900.8705 Validation Accuracy: 0.746094
Epoch  2,Loss:   956.2587 Validation Accuracy: 0.820312
Epoch  3,Loss:   422.5016 Validation Accuracy: 0.847656
Epoch  4,Loss:   534.4116 Validation Accuracy: 0.847656
Epoch  5,Loss:   346.2297 Validation Accuracy: 0.867188
Epoch  6,Loss:   490.7515 Validation Accuracy: 0.871094
Epoch  7,Loss:   191.4990 Validation Accuracy: 0.875000
Epoch  8,Loss:   345.5573 Validation Accuracy: 0.875000
Epoch  9,Loss:   258.5102 Validation Accuracy: 0.875000
Epoch 10,Loss:   246.9955 Validation Accuracy: 0.875000
Testing Accuracy: 0.890625


In [22]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
conv1 = conv2d(x, weights['wc1'], biases['bc1'])
print(conv1)
conv1 = maxpool2d(conv1, k = 2)
print(conv1),print()

conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
print(conv2)
conv2 = maxpool2d(conv2, k = 2)
print(conv2),print()

fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
print(fc1)
fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
print(fc1)
fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.dropout(fc1, dropout)
print(fc1),print()
out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
print(out)

Tensor("Relu_20:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool_13:0", shape=(?, 14, 14, 32), dtype=float32)

Tensor("Relu_21:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_14:0", shape=(?, 7, 7, 64), dtype=float32)

Tensor("Reshape_4:0", shape=(?, 3136), dtype=float32)
Tensor("Add_3:0", shape=(?, 1024), dtype=float32)
Tensor("dropout_2/mul:0", shape=(?, 1024), dtype=float32)

Tensor("Add_4:0", shape=(?, 10), dtype=float32)
